## Integration of Postgres with Pandas
Pandas is a powerful Python Data Library which is used to process as well as analyze the data. It have robust APIs to work with databases.

Here are the steps involved to use Pandas to work with databases like Postgres.
* We need to make sure `pandas`, `psycopg2-binary` as well as `sqlalchemy` installed using `pip`.
* Pandas uses `sqlalchemy` to interact with database tables based on the connection url.
* `sqlalchemy` is the most popular ORM to hide the complexity of connecting to the databases using our applications.
* Here are the examples using Pandas. 
  * Read from file, Process and Write to Postgres Database Table using Pandas. 
  * Read from Postgres Database Table using Pandas for the validation.

In [1]:
!pip install psycopg2-binary

You should consider upgrading via the 'D:\Project\Data Engineer\data-engineering-on-gcp\deg-venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install pandas

You should consider upgrading via the 'D:\Project\Data Engineer\data-engineering-on-gcp\deg-venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install sqlalchemy

     ---------------------------------------- 2.1/2.1 MB 8.3 MB/s eta 0:00:00
     -------------------------------------- 290.8/290.8 KB 9.1 MB/s eta 0:00:00


You should consider upgrading via the 'D:\Project\Data Engineer\data-engineering-on-gcp\deg-venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
import pandas as pd

In [5]:
columns = ['order_id', 'order_date', 'order_customer_id', 'order_status']

In [6]:
import json

In [7]:
schemas = json.load(open('../../data/retail_db/schemas.json'))

In [8]:
sorted(schemas['orders'], key=lambda col: col['column_position'])

[{'column_name': 'order_id', 'data_type': 'integer', 'column_position': 1},
 {'column_name': 'order_date', 'data_type': 'string', 'column_position': 2},
 {'column_name': 'order_customer_id',
  'data_type': 'timestamp',
  'column_position': 3},
 {'column_name': 'order_status', 'data_type': 'string', 'column_position': 4}]

In [9]:
columns = [col['column_name'] for col in sorted(schemas['orders'], key=lambda col: col['column_position'])]

In [10]:
columns

['order_id', 'order_date', 'order_customer_id', 'order_status']

In [11]:
pd.read_csv?

Signature:
pd.read_csv(
    filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]',
    *,
    sep: 'str | None | lib.NoDefault' = <no_default>,
    delimiter: 'str | None | lib.NoDefault' = None,
    header: "int | Sequence[int] | None | Literal['infer']" = 'infer',
    names: 'Sequence[Hashable] | None | lib.NoDefault' = <no_default>,
    index_col: 'IndexLabel | Literal[False] | None' = None,
    usecols: 'UsecolsArgType' = None,
    dtype: 'DtypeArg | None' = None,
    engine: 'CSVEngine | None' = None,
    converters: 'Mapping[Hashable, Callable] | None' = None,
    true_values: 'list | None' = None,
    false_values: 'list | None' = None,
    skipinitialspace: 'bool' = False,
    skiprows: 'list[int] | int | Callable[[Hashable], bool] | None' = None,
    skipfooter: 'int' = 0,
    nrows: 'int | None' = None,
    na_values: 'Hashable | Iterable[Hashable] | Mapping[Hashable, Iterable[Hashable]] | None' = None,
    keep_default_na: 'bool' = True,
    na_filter: '

In [ ]:
orders = pd.read_csv('../../data/retail_db/orders/part-00000', names=columns)

In [ ]:
orders

In [ ]:
type(orders)

In [ ]:
orders.shape

In [ ]:
daily_status_count = orders. \
    groupby(['order_date', 'order_status'])['order_id']. \
    agg(order_count='count'). \
    reset_index()

In [ ]:
daily_status_count

In [ ]:
daily_status_count.to_sql?

In [ ]:
host = '104.197.141.244'
port = 5432
database = 'itversity_retail_db'
user = 'itversity_retail_user'
password = 'itversity'

In [ ]:
conn_uri = f'postgresql://{user}:{password}@{host}:{port}/{database}'

In [ ]:
daily_status_count.to_sql(
    'daily_status_count',
    conn_uri,
    if_exists='replace',
    index=False
)

In [ ]:
pd.read_sql?

In [ ]:
dsc = pd.read_sql(
    'daily_status_count',
    conn_uri
)

In [ ]:
dsc.shape

In [ ]:
dsc

In [ ]:
dsc = pd.read_sql(
    '''
        SELECT order_status, sum(order_count) AS order_count FROM daily_status_count
        GROUP BY 1
        ORDER BY 2 DESC
    ''',
    conn_uri
)

In [ ]:
dsc